In [82]:
import torch 
import torchvision
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F

In [3]:
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=None)

0it [00:00, ?it/s]

9920512it [05:00, 31979.26it/s]                             

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



0it [00:00, ?it/s]


  0%|          | 0/28881 [00:00<?, ?it/s]
32768it [00:00, 98459.80it/s]            
0it [00:00, ?it/s]

...


 37%|███▋      | 606208/1648877 [00:18<00:30, 34136.16it/s]
9920512it [05:20, 31979.26it/s] [00:18<00:31, 32549.33it/s]
 65%|██████▌   | 1073152/1648877 [00:32<00:16, 34634.85it/s]


100%|█████████▉| 1646592/1648877 [00:49<00:00, 33881.23it/s]
1654784it [00:49, 31412.08it/s]                             

0it [00:00, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw




  0%|          | 0/4542 [00:00<?, ?it/s]

8192it [00:00, 36464.51it/s]            

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


In [4]:
mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=None)


1654784it [01:08, 31412.08it/s]

In [41]:
def process_data_set_binary_classif(data, targets, classes=[1,8]):
    
    assert (type(classes)) == list  and (len(classes)==2), "classes must be a list of two elements"
    number_interesting_labels = torch.sum(mnist_testset.targets == classes[0]).item() + torch.sum(mnist_testset.targets == classes[1]).item()
    binary_labels = torch.empty(number_interesting_labels)
    binary_data = torch.empty(number_interesting_labels, 28, 28)
    
    for i, t in enumerate(targets):
        c=0
        if t in classes:
            binary_labels[c] = t
            binary_data[c, :,:] = data[i]
            c += 1
            
    return binary_data, binary_labels
    

In [46]:
data_train, targets_train = process_data_set_binary_classif(mnist_trainset.data, mnist_trainset.targets)
data_test ,targets_test = process_data_set_binary_classif(mnist_testset.data, mnist_testset.targets)

In [98]:
class CnnModel(nn.Module):
    def __init__(self):
        super(CnnModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 1, (5, 5), bias=True)
        self.flatt = nn.Flatten()
        self.dense1 = nn.Linear(576, 1)
        self.dense2 = nn.Linear(576, 1)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.flatt(x)
        x1 = self.dense1(x)
        x2 = self.dense2(x)
        #return(x1)
        return F.softmax(torch.cat((x1, x2), dim=1).reshape(x.shape[0], 2), dim=1)


In [99]:
net = CnnModel()

In [100]:
net(data_train[0:20].reshape(20,1,28,28))

tensor([[0.9934, 0.0066],
        [   nan,    nan],
        [0.0000, 1.0000],
        [   nan,    nan],
        [1.0000, 0.0000],
        [   nan,    nan],
        [0.0000, 1.0000],
        [0.0000, 1.0000],
        [   nan,    nan],
        [1.0000, 0.0000],
        [   nan,    nan],
        [   nan,    nan],
        [0.0000, 1.0000],
        [0.0000, 1.0000],
        [   nan,    nan],
        [   nan,    nan],
        [1.0000, 0.0000],
        [   nan,    nan],
        [   nan,    nan],
        [   nan,    nan]], grad_fn=<SoftmaxBackward>)